## Orchestrating using Google Kubernetes Engine


**Note: While exploring GKE, keep a tab on billing (check every so often)!** 


### Introduction to Google Kubernetes Engine by GCP

 - Google Kubernetes Engine (GKE) by GCP a managed service for running K8s, with key features such as security, scaling and multi-cluster support taken care of as part of K8s on their infrastructure.

 - GKE's operation is very similar to ECS.

 - Our **goal** will be to use GKE for deploying our recommendation system (the ML model we have been using).
 	- We will first save our docker image to a Docker registry on GCP (this is called the Container Registry).
 	- Next we will use that image while setting up a K8s cluster.




### Google Container Registry

 - We will use the docker login command with the previously created service account with the JSON based credentials we had saved.

```bash
(base) ttmac:~ theja$ cat model-user.json | docker login -u _json_key --password-stdin https://us.gcr.io
Login Succeeded
```

 - Tag the docker image with the Google container registry specific tag as follows:

```bash
(base) ttmac:~ theja$ docker tag prediction_service us.gcr.io/authentic-realm-276822/prediction_service
(base) ttmac:~ theja$ docker images
REPOSITORY                                                     TAG                 IMAGE ID            CREATED             SIZE
prediction_service                                             latest              dd408a931e14        7 days ago          2.06GB
us.gcr.io/authentic-realm-276822/prediction_service            latest              dd408a931e14        7 days ago          2.06GB
weather_service                                                latest              20d340f941c0        9 days ago          496MB
debian                                                         buster-slim         c7346dd7f20e        6 weeks ago         69.2MB
continuumio/miniconda3                                         latest              b4adc22212f1        6 months ago        429MB
hello-world                                                    latest              bf756fb1ae65        8 months ago        13.3kB
```



 - Next, we push the local image to GCR. The upload status will keep getting updated.

```bash
(base) ttmac:~ theja$ docker push us.gcr.io/authentic-realm-276822/prediction_service
The push refers to repository [us.gcr.io/authentic-realm-276822/prediction_service]
d4bf100b2f89: Pushed
6719394c8842: Pushed
a432b6ec80f7: Pushing [>                                                  ]  20.81MB/1.635GB
fcd8d39597dd: Pushing [========>                                          ]  24.11MB/149.1MB
875120aa853c: Pushing [=====>                                             ]  23.17MB/210.4MB
f2cb0ecef392: Layer already exists
```
 - When its done, you will see the following:

```bash
(base) ttmac:~ theja$ docker push us.gcr.io/authentic-realm-276822/prediction_service
The push refers to repository [us.gcr.io/authentic-realm-276822/prediction_service]
d4bf100b2f89: Pushed
6719394c8842: Pushed
a432b6ec80f7: Pushed
fcd8d39597dd: Pushed
875120aa853c: Pushed
f2cb0ecef392: Layer already exists
latest: digest: sha256:f5b19d0e4510194ab8bdbed22f915fec8a07d1a465725ccfa6196782a480172c size: 1582
```



 - We can verify that the prediction_service image is present in the GCR page.

![gcr_image1](images/gcr_image1.png)

![gcr_image2](images/gcr_image2.png)




### Google Kubernetes Engine

 - We will now set up the K8s cluster. Lets start by accessing the GKE page.


![gke01](images/gke01.png)

 - Click on `Deply container` next to the `Create cluster` button.

![gke02](images/gke02.png)

 - Pick the existing cluster option and choose `select`.

![gke03](images/gke03.png)

 - Choose the recently uploaded prediction_service image.

 ![gke04](images/gke04.png)

 - Hit continue.

 ![gke05](images/gke05.png)



 - On the next page, we will leave everything to default except for the name and then hit `Deploy` button.

 ![gke06](images/gke06.png)
 ![gke07](images/gke07.png)

 - It may take some time for the cluster to get fully set up.

 ![gke08](images/gke08.png)

  - Recall the system level diagram.

 	![kubernetes_cluster](images/kubernetes_cluster.svg)
	<div style="text-align: right"> Source: https://www.gstatic.com/pantheon/images/container/kubernetes_cluster.svg </div>



  - Once the cluster is set up, we can investigate its properties.


 ![gke09](images/gke09.png)  

 ![gke10](images/gke10.png)  

 ![gke11](images/gke11.png)  

 ![gke12](images/gke12.png)  



 - Just as we did in the local deployment, we will expose the cluster to be able to trigger prediction requests. We can do that by going to the Workload tab and clicking on the cluster.

 ![gke13](images/gke13.png)

 - We will next click on the `expose` button to the far right.

 ![gke14](images/gke14.png)



 - We will specify the container port as 80 (if you look at recommend.py we have specified port 80 where the flask app listens to requests).

 ![gke15](images/gke15.png)

 - Once the service is running, we can obtain the external IP.

  ![gke16](images/gke16.png)



 - As expected, if we query without a payload we get a default response.

  ![gke_browser1](images/gke_browser1.png)


 - With an example payload, we are able to retrieve the recommendations from real-time execution of the pytorch model.

  ![gke_browser2](images/gke_browser2.png)



 - To tear down the cluster, we first delete the service.


![teardown_service1](images/teardown_service1.png)

![teardown_service2](images/teardown_service2.png)

![teardown_service3](images/teardown_service3.png)



 - Finally, we can delete the workload itself.

![teardown_workload1](images/teardown_workload1.png)
![teardown_workload2](images/teardown_workload2.png)

